In [1]:
import pygsheets
from tqdm import tqdm

In [2]:
client = pygsheets.authorize()

### Indonesian Data

In [44]:
id_to_key_ref_ind = {
    'W1': '14dAiIsBLxBUHzXgFDznMjSXCJmFY6qnIWOADqITWx9E',
    'W2': '1jHWRjwkahxpA5T_BWBRaMQ4q9kyg1l0gGz42pC6-dxI',
    'W3': '1C-Yyc17WFWScC5eu5WttQc8P8vCFuLvh8MqlBKiwF8c',
    'W4': '1J0xqeC05H1RVPSERi8fCXWUTGJ4iH3SI0w59BoFh_KU',
    'W5': '1T9jbfP1iapNLS94QZMiYw5K6LrZoP01Qf1jvTWpvdkU',
    'W6': '1G0chmXUmWC-lrZLg9ToQTGCdQmnbP0DA204bIKJn9yc',
}
id_to_key_pred_ind = {
    'W1': '1LIVvmdGV6LHEZDxJjYVre9EkMwJ48tp6mpBPaz7LWoE',
    'W2': '1noJV_ViNfUGabWyWwGChEsVhfPc4ctxh2oVY9kcUaO4',
    'W3': '1M9QwUbbiHkZl_13Y-CNrfzNYN9No5hp_u6KWYhZdi1k',
    'W4': '1gz9l9wsY84cgtKJcYRCCuto7piBpv0DWY2jUDxsndI0',
    'W5': '1iPZV_yL3qwKmSi4mtDwhZNZlB90a63SoFRs2bChVeYc',
    'W6': '1YrprgKaeWU0L4jq78ioL40TyxEC0_nwGroEJVLqhjcg',
}

In [45]:
pred_id = 'W6'
ref_ids = ['W1', 'W2', 'W3', 'W4', 'W5']

sh_pred = client.open_by_key(id_to_key_pred_ind[pred_id])

for ref_id in ref_ids:
    print(f'Reading {ref_id}...')
    wks_pred = sh_pred.worksheet('title', ref_id)
    sh_ref = client.open_by_key(id_to_key_ref_ind[ref_id])
    wks_ref = sh_ref.worksheet('title', 'Data')

    # Get answers
    ref_ans = wks_ref.get_col(6)[1:]
    pred_ans = wks_pred.get_col(6)[1:]

    # Check answer consistency
    is_consistent = []
    const_temp = []
    rows_idx = []
    idx = 1
    for pred, ref in tqdm(zip(pred_ans, ref_ans)):
        idx += 1
        consistency = 'TRUE' if pred == ref else 'FALSE'
        const_temp.append(consistency)

        if len(const_temp) == 5:
            if 'FALSE' in const_temp:
                const_temp[0] = 'FALSE'
            for const in const_temp:
                is_consistent.append(const)
            const_temp = []
            rows_idx.append(('H' + str(idx-4), 'H' + str(idx)))

    # Add new colomn & update its values
    wks_pred.insert_cols(6, 2)
    wks_pred.adjust_column_width(7, 8, pixel_size=100)
    wks_pred.set_data_validation('G2', 'G1251', condition_type='BOOLEAN')
    wks_pred.set_data_validation('H2', 'H1251')
    wks_pred.update_col(7, ['Gold Answer'] + ref_ans)
    wks_pred.update_col(8, ['Is Consistent'] + is_consistent)

    wks_pred.add_conditional_formatting(start='H2', 
                                     end='H1251', 
                                     condition_type='TEXT_CONTAINS', 
                                     format={'backgroundColor': {'red': 0.957, 'green': 0.8, 'blue': 0.8, 'alpha': 1}}, 
                                     condition_values=['FALSE'])

    requests = []
    for start, end in tqdm(rows_idx):
        requests.append({
            'mergeCells': {
                'range': {
                    'sheetId': wks_pred.id,
                    'startRowIndex': int(start[1:]) - 1,
                    'endRowIndex': int(end[1:]),
                    'startColumnIndex': ord(start[0].upper()) - ord('A'),
                    'endColumnIndex': ord(end[0].upper()) - ord('A') + 1
                },
                'mergeType': 'MERGE_ALL'
            }
        })

    # Perform the batch update
    wks_pred.client.sheet.batch_update(sh_pred.id, requests)

Reading W1...


1250it [00:00, 369086.94it/s]
100%|██████████| 250/250 [00:00<00:00, 146941.70it/s]


Reading W2...


1250it [00:00, 367380.00it/s]
100%|██████████| 250/250 [00:00<00:00, 120692.45it/s]


Reading W3...


1250it [00:00, 308913.50it/s]
100%|██████████| 250/250 [00:00<00:00, 135335.05it/s]


Reading W4...


1250it [00:00, 361952.36it/s]
100%|██████████| 250/250 [00:00<00:00, 111776.57it/s]


Reading W5...


1250it [00:00, 376806.09it/s]
100%|██████████| 250/250 [00:00<00:00, 127007.75it/s]


### Sundanese Data

In [57]:
id_to_key_ref_sun = {
    'W1': '1R-bT3RMu41cx-fnac3mNdb3OT_kb6vof1eMdCbU8_jM',
    'W2': '1ugtKZhO4jLtxW_yEwrrPXZtry2pLaP-6QgiIHKldl5E',
    'W3': '1OzP1XwzU3c-rNXyqxU3JKrrXHeXzSu3D6FIcfym4fHE',
    'W4': '1dNwW9dL4YPBEypUHQ6_-Dse8XL851W0WUiCBb28hRBQ',
    'W5': '16wXGP08nESdLg4am4wfngeMNbK81IJAo3p0udg_iVlU',
    'W6': '1ZQEBOPGAlc6f2IUwavEKpPHY7dQxV4lN5iSQm1WHnX4',
}
id_to_key_pred_sun = {
    'W1': '1MD7saLQYAXCLnpTA_WaV1oVv2vwbBn5hFfJ3z1s-Npg',
    'W2': '1Tn6nttAU3wAiMaJaBb6-P39IJ2sElMNS09bXUo-vI-Q',
    'W3': '1AQG9V74TxDkQAUt5aRs9mNa2fiyLMIML65X8nNQ-WuU',
    'W4': '1vSIJikiPedE4GNUB-zno2SN-L7c_e0IJMcrP8nq2RFk',
    'W5': '1b7G4c2PejzDpSqr4Nxo9sWYiAuE_egJS35Dt1HUkDvY',
    'W6': '1nu3ksVRHl-bTO0LW6yXIHGNhs7iJJ5AsdmHcrb2KxL0',
}

In [58]:
pred_id = 'W6'
ref_ids = ['W1', 'W2', 'W3', 'W4', 'W5']

sh_pred = client.open_by_key(id_to_key_pred_sun[pred_id])

for ref_id in ref_ids:
    print(f'Reading {ref_id}...')
    wks_pred = sh_pred.worksheet('title', ref_id)
    sh_ref = client.open_by_key(id_to_key_ref_sun[ref_id])
    wks_ref = sh_ref.worksheet('title', 'Data')

    # Get answers
    ref_ans = wks_ref.get_col(7)[1:]
    pred_ans = wks_pred.get_col(7)[1:]

    # Check answer consistency
    is_consistent = []
    const_temp = []
    rows_idx = []
    idx = 1
    for pred, ref in tqdm(zip(pred_ans, ref_ans)):
        idx += 1
        consistency = 'TRUE' if pred == ref else 'FALSE'
        const_temp.append(consistency)

        if len(const_temp) == 5:
            if 'FALSE' in const_temp:
                const_temp[0] = 'FALSE'
            for const in const_temp:
                is_consistent.append(const)
            const_temp = []
            rows_idx.append(('I' + str(idx-4), 'I' + str(idx)))

    # Add new colomn & update its values
    wks_pred.insert_cols(7, 2)
    wks_pred.adjust_column_width(8, 9, pixel_size=100)
    wks_pred.set_data_validation('H2', 'H1251', condition_type='BOOLEAN')
    wks_pred.set_data_validation('I2', 'I1251')
    wks_pred.update_col(8, ['Gold Answer'] + ref_ans)
    wks_pred.update_col(9, ['Is Consistent'] + is_consistent)

    wks_pred.add_conditional_formatting(start='I2', 
                                        end='I1251', 
                                        condition_type='TEXT_CONTAINS', 
                                        format={'backgroundColor': {'red': 0.957, 'green': 0.8, 'blue': 0.8, 'alpha': 1}}, 
                                        condition_values=['FALSE'])

    requests = []
    for start, end in tqdm(rows_idx):
        requests.append({
            'mergeCells': {
                'range': {
                    'sheetId': wks_pred.id,
                    'startRowIndex': int(start[1:]) - 1,
                    'endRowIndex': int(end[1:]),
                    'startColumnIndex': ord(start[0].upper()) - ord('A'),
                    'endColumnIndex': ord(end[0].upper()) - ord('A') + 1
                },
                'mergeType': 'MERGE_ALL'
            }
        })

    # Perform the batch update
    wks_pred.client.sheet.batch_update(sh_pred.id, requests)

Reading W1...


1250it [00:00, 365739.80it/s]
100%|██████████| 250/250 [00:00<00:00, 119291.92it/s]


Reading W2...


1250it [00:00, 329512.92it/s]
100%|██████████| 250/250 [00:00<00:00, 121785.83it/s]


Reading W3...


1250it [00:00, 372310.75it/s]
100%|██████████| 250/250 [00:00<00:00, 108357.55it/s]


Reading W4...


1250it [00:00, 354320.47it/s]
100%|██████████| 250/250 [00:00<00:00, 115177.50it/s]


Reading W5...


1250it [00:00, 376292.26it/s]
100%|██████████| 250/250 [00:00<00:00, 120387.60it/s]
